In [1]:
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [2]:
# 데이터 분할(폴더) 구분
data_splits = ["train", "test"]

# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}


for split in data_splits:
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]
        
        file_path = f"./data/{folder}/{split}_{suffix}.parquet"
        # 변수명 형식: {var_prefix}_{split}_{month}
        variable_name = f"{var_prefix}_{split}"
        globals()[variable_name] = pd.read_parquet(file_path, engine="pyarrow")
        print(f"{variable_name} is loaded from {file_path}")

gc.collect()

customer_train is loaded from ./data/1.회원정보/train_회원정보.parquet
credit_train is loaded from ./data/2.신용정보/train_신용정보.parquet
sales_train is loaded from ./data/3.승인매출정보/train_승인매출정보.parquet
billing_train is loaded from ./data/4.청구입금정보/train_청구정보.parquet
balance_train is loaded from ./data/5.잔액정보/train_잔액정보.parquet
channel_train is loaded from ./data/6.채널정보/train_채널정보.parquet
marketing_train is loaded from ./data/7.마케팅정보/train_마케팅정보.parquet
performance_train is loaded from ./data/8.성과정보/train_성과정보.parquet
customer_test is loaded from ./data/1.회원정보/test_회원정보.parquet
credit_test is loaded from ./data/2.신용정보/test_신용정보.parquet
sales_test is loaded from ./data/3.승인매출정보/test_승인매출정보.parquet
billing_test is loaded from ./data/4.청구입금정보/test_청구정보.parquet
balance_test is loaded from ./data/5.잔액정보/test_잔액정보.parquet
channel_test is loaded from ./data/6.채널정보/test_채널정보.parquet
marketing_test is loaded from ./data/7.마케팅정보/test_마케팅정보.parquet
performance_test is loaded from ./data/8.성과정보/test_성과정보.parquet


0

In [3]:
# 데이터 유형별 설정 
info_categories = ["customer", "credit", "sales", "billing", "balance", "channel", "marketing", "performance"]

In [4]:
#### Train ####

# 각 유형별로 월별 데이터를 합쳐서 새로운 변수에 저장
train_dfs = {}

for prefix in info_categories:
    # globals()에서 동적 변수명으로 데이터프레임들을 가져와 리스트에 저장
    df_list = [globals()[f"{prefix}_train"]]
    train_dfs[f"{prefix}_train_df"] = pd.concat(df_list, axis=0)
    gc.collect()
    print(f"{prefix}_train_df is created with shape: {train_dfs[f'{prefix}_train_df'].shape}")
    print(train_dfs[f'{prefix}_train_df'].head(1))


customer_train_df = train_dfs["customer_train_df"]
credit_train_df   = train_dfs["credit_train_df"]
sales_train_df    = train_dfs["sales_train_df"]
billing_train_df  = train_dfs["billing_train_df"]
balance_train_df  = train_dfs["balance_train_df"]
channel_train_df  = train_dfs["channel_train_df"]
marketing_train_df= train_dfs["marketing_train_df"]
performance_train_df = train_dfs["performance_train_df"]

gc.collect()

customer_train_df is created with shape: (400000, 78)
     기준년월            ID  남녀구분코드   연령 Segment  회원여부_이용가능  회원여부_이용가능_CA  \
0  201811  TRAIN_000000       2  40대       D          1             1   

   회원여부_이용가능_카드론  소지여부_신용  소지카드수_유효_신용  ...  할인금액_제휴연회비_B0M  청구금액_기본연회비_B0M  \
0              0        1            1  ...               0               0   

   청구금액_제휴연회비_B0M  상품관련면제카드수_B0M  임직원면제카드수_B0M  우수회원면제카드수_B0M  기타면제카드수_B0M  \
0               0             0개            0개             0개           0개   

   카드신청건수  Life_Stage  최종카드발급경과월  
0       0     자녀성장(2)         26  

[1 rows x 78 columns]
credit_train_df is created with shape: (400000, 42)
     기준년월            ID  최초한도금액  카드이용한도금액  CA한도금액  일시상환론한도금액  월상환론한도금액  \
0  201811  TRAIN_000000       0     19580    5816          0         0   

   CA이자율_할인전  CL이자율_할인전  RV일시불이자율_할인전  ...  연체감액여부_R3M  한도심사요청건수  한도요청거절건수  \
0  22.996044  18.557129      18.27795  ...           0        0회         0   

   한도심사요청후경과월 한도심사거절후경과월  시장단기연체

0

In [5]:
from functools import reduce

# 공통 ID 컬럼명
COMMON_ID = 'ID' 
# 삭제할 중복 컬럼명
CONFLICT_COL = '기준년월' 

# DF
data_to_merge = [
    customer_train_df,
    credit_train_df,
    sales_train_df,
    billing_train_df,
    balance_train_df,
    channel_train_df,
    marketing_train_df,
    performance_train_df
]

all_train_dfs_processed = []

print(f"--- Dropping '{CONFLICT_COL}' column before merge ---")

# 1. 각 DF에서 COMMON_ID가 아닌 '기준년월' 컬럼 삭제
for i, df in enumerate(data_to_merge):
    df_processed = df.copy()
    
    # 해당 컬럼이 존재하고, ID 컬럼이 아닌 경우에만 삭제
    if CONFLICT_COL in df_processed.columns and CONFLICT_COL != COMMON_ID:
        df_processed = df_processed.drop(columns=[CONFLICT_COL])
        print(f"Dropped '{CONFLICT_COL}' from DataFrame {i+1}")
    
    all_train_dfs_processed.append(df_processed)

# 2. '기준년월'이 삭제된 DF 리스트로 reduce merge 실행
print("\nStarting merge...")
merged_train_df = reduce(
    lambda left, right: pd.merge(left, right, on=COMMON_ID, how='left'), 
    all_train_dfs_processed
)

print(f"--- Merge successful! ---")
print(f"최종 병합된 데이터 Shape: {merged_train_df.shape}")

--- Dropping '기준년월' column before merge ---
Dropped '기준년월' from DataFrame 1
Dropped '기준년월' from DataFrame 2
Dropped '기준년월' from DataFrame 3
Dropped '기준년월' from DataFrame 4
Dropped '기준년월' from DataFrame 5
Dropped '기준년월' from DataFrame 6
Dropped '기준년월' from DataFrame 7
Dropped '기준년월' from DataFrame 8

Starting merge...
--- Merge successful! ---
최종 병합된 데이터 Shape: (400000, 857)


In [ ]:
# (이산형, 수치형) 데이터로 나누기 
target_col = 'Segment'
id_col = ['customer_id']

features_df = merged_train_df.drop(columns=[target_col] + id_col, errors='ignore')

Discrimination_criteria = 30 # 30 개를 기준으로 이산형, 수치형으로 분류

# object, categorical은 미리 이산형으로 분류
initial_categorical = features_df.select_dtypes(include=['object', 'category']).columns.tolist()
initial_numerical = features_df.select_dtypes(include=np.number).columns.tolist()

refined_numerical_features = []
refined_categorical_features = list(initial_categorical) 

# 수치형 피처들을 다시 검토
for col in initial_numerical:
    if features_df[col].nunique() < Discrimination_criteria:
        refined_categorical_features.append(col)
    else:
        refined_numerical_features.append(col)

print(f"--- 정제된 피처 분리 결과 (임계값: {Discrimination_criteria}) ---")
print(f"수치형 피처: {len(refined_numerical_features)}개")
print(f"범주형 피처: {len(refined_categorical_features)}개")




--- 정제된 피처 분리 결과 (임계값: 30) ---
수치형 피처: 351개
범주형 피처: 505개


In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.feature_selection import f_classif, chi2
import warnings

# --- 0. 타겟 변수(y) 인코딩 ---
# (★) 타겟 변수명을 확인하세요.
y = merged_train_df[target_col]

# 'Segment' (A~E)를 숫자(0~4)로 변환
le = LabelEncoder()
y_encoded = le.fit_transform(y)


# --- 1. 수치형 피처 (Numerical) → ANOVA F-test ---

print("\n--- 1. 수치형 피처 vs. 타겟 (ANOVA F-test) ---")

# 결측치를 0으로 단순 대체 (추후 더 나은 imputation 방법 고려)
X_num = features_df[refined_numerical_features].fillna(0)


# ANOVA F-test 실행
f_scores, p_values = f_classif(X_num, y_encoded)

# 결과를 DataFrame으로 정리
num_results_df = pd.DataFrame({
    'Feature': refined_numerical_features,
    'F_Score': f_scores,
    'P_Value': p_values
})

# P_Value가 낮은 순(연관성이 높은 순)으로 정렬
num_results_df = num_results_df.sort_values(by='P_Value', ascending=True)
num_results_df = num_results_df.sort_values(by='F_Score', ascending=False)


print("타겟과 연관성 높은 수치형 피처 TOP 50:")
print(num_results_df)


# --- 2. 범주형 피처 (Categorical) → Chi-Squared ---

print("\n--- 2. 범주형 피처 vs. 타겟 (Chi-Squared) ---")

# (★) 모든 범주형 피처를 '문자열'로 변환하고 결측치를 'Missing'으로 대체
X_cat = features_df[refined_categorical_features].astype(str).fillna('Missing')

# 범주형 피처들을 숫자로 인코딩 (Chi-Squared는 숫자 입력만 받음)
encoder = OrdinalEncoder()
X_cat_encoded = encoder.fit_transform(X_cat)

# Chi-Squared 테스트 실행
chi_scores, p_values = chi2(X_cat_encoded, y_encoded)

# 결과를 DataFrame으로 정리
cat_results_df = pd.DataFrame({
    'Feature': refined_categorical_features,
    'Chi2_Score': chi_scores,
    'P_Value': p_values
})

# P_Value가 낮은 순(연관성이 높은 순)으로 정렬
cat_results_df = cat_results_df.sort_values(by='P_Value', ascending=True)
cat_results_df = cat_results_df.sort_values(by='Chi2_Score', ascending=False)

print("타겟과 연관성 높은 범주형 피처 TOP 50:")
print(cat_results_df)


--- 1. 수치형 피처 vs. 타겟 (ANOVA F-test) ---
타겟과 연관성 높은 수치형 피처 TOP 50:
               Feature       F_Score   P_Value
207         정상청구원금_B5M  73956.952736  0.000000
199         정상청구원금_B0M  70749.746063  0.000000
203         정상청구원금_B2M  69233.371386  0.000000
3        이용금액_R3M_신용체크  66105.622588  0.000000
215           청구금액_R6M  57040.790862  0.000000
..                 ...           ...       ...
253        연체잔액_할부_B0M      1.663632  0.155313
146  할부금액_무이자_14M_R12M      1.039589  0.385000
251           연체잔액_B0M      0.908552  0.457768
277        연체잔액_CA_B0M      0.589492  0.670236
228      포인트_잔여포인트_B0M      0.495042  0.739407

[351 rows x 3 columns]

--- 2. 범주형 피처 vs. 타겟 (Chi-Squared) ---
타겟과 연관성 높은 범주형 피처 TOP 50:
              Feature     Chi2_Score  P_Value
250      이용건수_온라인_B0M  369574.427008      0.0
253   이용건수_페이_온라인_R6M  327784.600098      0.0
276     이용건수_간편결제_R3M  289550.492648      0.0
188      할부건수_6M_R12M  231339.126263      0.0
255   이용건수_페이_온라인_R3M  204497.995482      0.0
.. 

In [ ]:
print('수치형 데이터의 높은 연관성 : (청구원금, 이용금액)')
print('이산형 데이터의 높은 연관성 : (온라인 이용, 페이(간편결제) 및 할부)')